In [1]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt

Была взята готовая НС архетектуры SSD

- deploy.prototxt — файл описывающий слои нашей нейронной сети.
- caffemodel — веса, вычисленные во время тренировки модели.

In [2]:
protoFile = "deploy.prototxt.txt"
weightsFile = "res10_300x300_ssd_iter_140000_fp16.caffemodel"

Загрузка модели

In [3]:
model = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

# Получение изображения с вебкамеры
В окне с названием "camera" будет отображаться видео. 
Окно будет закрыто после нажатия Esc.

In [4]:
#Захватываем веб камеру. 0 — это индекс камеры, если их несколько то будет 0 или 1 и т.д.
cap = cv2.VideoCapture(0)
while True:
    # Читаем с устройства кадр(картинку) , метод возвращает флаг ret (True , False) и 
    # imgage — саму картинку ( массив numpy) .
    ret, image = cap.read()
    # В окне "camera" будут отображаться кадры с камеры
    cv2.imshow("camera", image)
    if cv2.waitKey(10) == 27: # Клавиша Esc
        break
cap.release()
cv2.destroyAllWindows()

По аналогии с распознованием одного изображения будет происходить распознования кадров видео. 

In [5]:
cap = cv2.VideoCapture(0)
while True:
    ret, image = cap.read()
    height, width = image.shape[:2]
    blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), (104.0, 177.0, 123.0))
    model.setInput(blob)
    output = np.squeeze(model.forward())
    for i in range(0, output.shape[0]):
        confidence = output[i, 2]
        if confidence > 0.5:
            box = output[i, 3:7] * np.array([width, height, width, height])
            start_x, start_y, end_x, end_y = box.astype(np.int)
            cv2.rectangle(image, (start_x, start_y), (end_x, end_y), color=(255, 0, 0), thickness=2)
            cv2.putText(image, f"{confidence*100:.2f}%", (start_x, start_y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    # show the image
    cv2.imshow("camera", image)
    if cv2.waitKey(10) == 27: 
        break
        
cap.release()
cv2.destroyAllWindows()